In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

plt.style.use("ggplot")
sns.set_theme()
pd.set_option("display.max_columns",None)

## Chargement du dataset et appercu general

In [ ]:
#charge le dataset
df = pd.read_csv("netflix_titles.csv")

print("Shape (lignes,colonnes) :",df.shape)
display(df.head())

df.info()

df.describe(include="all").T

## Valeurs manquantes et doublons

In [ ]:
#Valeurs manquantes par colonnes
df.isna().sum()

In [ ]:
#pourcentage de valeurs manquantes par colonnes
(df.isna().mean()*100).round(2)

In [ ]:
#Doublons
print("Nombres de lignes dupliquee :",df.duplicated().sum())

## Nettoyage

In [ ]:
#S'occupe des dates pas formatte pareill.
#Split en 2 colonnes; date added en datetime , year added et month added.
#transofrme aussi date_added en datetime pour faciliter les analyses
df["date_added"] = pd.to_datetime(df["date_added"],
    format="mixed"
    )
df["year_added"] = df["date_added"].dt.year
df["month_added"] = df["date_added"].dt.month

print(df[["date_added", "year_added", "month_added"]].head())

In [ ]:
#extrait la partie int de la duration.
df["duration_int"] = (
    df["duration"].str.extract(r"(\d+)").astype(float)
)

#visualisation et check
df[["type","duration","duration_int"]].head()

In [ ]:
display(df.head())

## analyse des contenus

### Films vs séries : proportions, tendances par année

In [ ]:
#Sort le count de movie et de tv show
df["type"].value_counts()

### Répartition des Movies / TV Shows

In [ ]:
plt.figure(figsize=(5,4))
sns.countplot(data=df,x="type")
plt.title("Répartition des contenus par type")
plt.xlabel("")
plt.ylabel("Nombres de titres")
plt.show()

### Évolutions par années

In [ ]:
type_year = (
    df
    .groupby(["release_year", "type"])
    .size()
    .reset_index(name="count")
    .sort_values("release_year")
)

display(type_year.head())

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(
    data=type_year,
    x="release_year",
    y="count",
    hue="type"
)
plt.title("Nombre de films / séries par année de sortie")
plt.xlabel("Année de sortie")
plt.ylabel("Nombre de titres")
plt.show()

### Genre principaux

In [ ]:
#Exploser les genre car il sont separé par des virgules
df_genres = df[["show_id", "type", "listed_in"]].dropna().copy()

df_genres["genre"] = df_genres["listed_in"].str.split(", ")
df_genres = df_genres.explode("genre")

df_genres.head()

In [ ]:
#Top 15 des genres toutes series et film confondu
top_genres = (
    df_genres["genre"]
    .value_counts()
    .head(15)
    .rename_axis("genre")      
    .reset_index(name="count")
)

display(top_genres)

In [ ]:
#plot du top 15 des genres
plt.figure(figsize=(8,6))
sns.barplot(data=top_genres, y="genre", x="count")
plt.title("Top 15 des genres les plus fréquents")
plt.xlabel("Nombre de titres")
plt.ylabel("Genre")
plt.show()

### Regroupement des genres

In [ ]:
#On vas essayer de faire un regroupement
df_genres = df[["show_id", "listed_in"]].dropna().copy()
df_genres["genre"] = df_genres["listed_in"].str.split(", ")
df_genres = df_genres.explode("genre")

df_genres["genre"] = df_genres["genre"].str.strip()
df_genres.head()

In [ ]:
#Essaie de regroupement par mots cle
def regrouper_genre(g):
    g_lower = g.lower()
    
    if "children" in g_lower or "family" in g_lower or "kids" in g_lower:
        return "Enfants / Famille"
    if "comed" in g_lower:
        return "Comédie"
    if "drama" in g_lower:
        return "Drame"
    if "horror" in g_lower:
        return "Horreur"
    if "action" in g_lower or "adventure" in g_lower:
        return "Action / Aventure"
    if "document" in g_lower or "docuseries" in g_lower:
        return "Documentaire"
    if "romantic" in g_lower or "romance" in g_lower:
        return "Romance"
    if "anime" in g_lower:
        return "Anime"
    if "thriller" in g_lower:
        return "Thriller"
    if "reality" in g_lower:
        return "Téléréalité"
    if "stand-up" in g_lower:
        return "Humour / Stand-up"
    if "international" in g_lower:
        return "International"
    
    return "Autre"

df_genres["genre_group"] = df_genres["genre"].apply(regrouper_genre)
df_genres[["genre", "genre_group"]].head(10)


In [ ]:
top_genre_groups = (
    df_genres["genre_group"]
    .value_counts()
    .rename_axis("genre_group")   
    .reset_index(name="count")    
)

display(top_genre_groups)

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=top_genre_groups, y="genre_group", x="count")
plt.title("Fréquences des grandes familles de genres")
plt.xlabel("Nombre de titres")
plt.ylabel("Famille de genre")
plt.show()

### Repartition geographique

### Casting & réalisateurs : analyse simple (comptages, noms fréquents)

## Analyse Temporelle

### Distribution par annee

### Analyse de la colonne date_added.

## Visualistion Interactive

### Représentations obligatoires :

### histogrammes, countplots, boxplots (Seaborn / Matplotlib)
### visualisations interactives (Plotly)

### Choix libres : nuages de mots, diagrammes circulaires, timelines, etc.

## Synthèse


### Résumé des observations principales.

### Tendances remarquables (contenus récents, pays dominants, genrespopulaires).
